In [1]:
import json
import os

from qwen_agent.llm import get_chat_model



In [2]:
print("你好")

你好


In [12]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location):
    """Get the current weather in a given location"""
    print("location:",location)
    if '北京' in location:
        return json.dumps({'location': '北京', 'temperature': '10', 'unit': '摄氏度'})
    elif '深圳' in location:
        return json.dumps({'location': '深圳', 'temperature': '21', 'unit': '摄氏度'})
    elif '上海' in location:
        return json.dumps({'location': '上海', 'temperature': '22', 'unit': '摄氏度'})
    else:
        return json.dumps({'location': location, 'temperature': 'unknown'})

In [14]:
result=get_current_weather("上海")

location: 上海


In [15]:
result

'{"location": "\\u4e0a\\u6d77", "temperature": "22", "unit": "\\u6444\\u6c0f\\u5ea6"}'

In [16]:
def test():
    llm = get_chat_model(
    
    {
    # 使用与 OpenAI API 兼容的模型服务，例如 vLLM 或 Ollama：
    'model': '/models/chatglm3-6b',
    'model_server': 'http://172.16.7.132:23148/v1',  # base_url，也称为 api_base
    # 'api_key': 'EMPTY',
    # "AMAP_TOKEN":""

    # （可选） LLM 的超参数：
    # 'generate_cfg': {
    #     'top_p': 0.8
    # }
}
    )

    # Step 1: send the conversation and available functions to the model
    messages = [{'role': 'user', 'content': "上海的天气怎么样"}]
    functions = [{
        'name': 'get_current_weather',
        'description': 'Get the current weather in a given location',
        'parameters': {
            'type': 'object',
            'properties': {
                'location': {
                    'type': 'string',
                    'description': '城市，例如：北京、上海',
                },
                'unit': {
                    'type': 'string',
                    'enum': ['celsius', 'fahrenheit']
                },
            },
            'required': ['location'],
        },
    }]

    print('# Assistant Response 1:')
    responses = []
    for responses in llm.chat(
            messages=messages,
            functions=functions,
            stream=True,
            # Note: extra_generate_cfg is optional
            # extra_generate_cfg=dict(
            #     # Note: if function_choice='auto', let the model decide whether to call a function or not
            #     # function_choice='auto',  # 'auto' is the default if function_choice is not set
            #     # Note: set function_choice='get_current_weather' to force the model to call this function
            #     function_choice='get_current_weather',
            # ),
    ):
        print(responses)

    messages.extend(responses)  # extend conversation with assistant's reply

    # Step 2: check if the model wanted to call a function
    last_response = messages[-1]
    if last_response.get('function_call', None):

        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            'get_current_weather': get_current_weather,
        }  # only one function in this example, but you can have multiple
        function_name = last_response['function_call']['name']
        function_to_call = available_functions[function_name]
        function_args = json.loads(last_response['function_call']['arguments'])
        function_response = function_to_call(
            location=function_args.get('location'),
            unit=function_args.get('unit'),
        )
        print('# Function Response:')
        print(function_response)

        # Step 4: send the info for each function call and function response to the model
        messages.append({
            'role': 'function',
            'name': function_name,
            'content': function_response,
        })  # extend conversation with function response

        print('# Assistant Response 2:')
        for responses in llm.chat(
                messages=messages,
                functions=functions,
                stream=True,
        ):  # get a new response from the model where it can see the function response
            print(responses)


In [17]:
test()

# Assistant Response 1:
[{'role': 'assistant', 'content': '上海的'}]
[{'role': 'assistant', 'content': '上海的天气'}]
[{'role': 'assistant', 'content': '上海的天气当前'}]
[{'role': 'assistant', 'content': '上海的天气当前是'}]
[{'role': 'assistant', 'content': '上海的天气当前是['}]
[{'role': 'assistant', 'content': '上海的天气当前是[f'}]
[{'role': 'assistant', 'content': '上海的天气当前是[fahren'}]
[{'role': 'assistant', 'content': '上海的天气当前是[fahrenheit'}]
[{'role': 'assistant', 'content': '上海的天气当前是[fahrenheit]'}]
[{'role': 'assistant', 'content': '上海的天气当前是[fahrenheit]气温'}]
[{'role': 'assistant', 'content': '上海的天气当前是[fahrenheit]气温，'}]
[{'role': 'assistant', 'content': '上海的天气当前是[fahrenheit]气温，也就是'}]
[{'role': 'assistant', 'content': '上海的天气当前是[fahrenheit]气温，也就是['}]
[{'role': 'assistant', 'content': '上海的天气当前是[fahrenheit]气温，也就是[c'}]
[{'role': 'assistant', 'content': '上海的天气当前是[fahrenheit]气温，也就是[cels'}]
[{'role': 'assistant', 'content': '上海的天气当前是[fahrenheit]气温，也就是[celsius'}]
[{'role': 'assistant', 'content': '上海的天气当前是[fahrenheit]气温，也就是[cel